In [57]:
import keras as ks
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pickle as pkl

In [2]:
from model import create_model
from preprocessing import preprocess

In [3]:
orig_labels=[]
with open("./labels.pickle","rb") as file:
    orig_labels=pkl.load(file)
    
aug_labels=[]
with open("./augments_label.pickle","rb") as file:
    aug_labels=pkl.load(file)

labels=orig_labels+aug_labels

# len(labels)

In [4]:
def simplify_labels(labels):
    new_labels=[]
    for image in labels:
        for num in range(image[1][0]):
            new_labels.append((image[0],(1,[image[1][1][num]],[image[1][2][num]])))
    return new_labels
labels=simplify_labels(labels)
np.random.shuffle(labels)
len(labels)

1920

In [5]:
total_data=0
ones=[]
for image in labels:
    total_data+=image[1][0]
    ones+=image[1][-1]
total_data,(np.array(ones)=='1').sum()

(1920, 786)

In [6]:
batch_size=256
epochs=2
steps_per_epoch=total_data/batch_size

In [11]:
model=create_model((224,224,3))
gen=preprocess(labels,"./Data/images/",batch_size=batch_size)

In [11]:
import keras as ks
# import tensorflow as tf

def create_model(input_shape):
    base_model=ks.applications.VGG16(include_top=False,input_shape=input_shape)

    trainable=False
    for layer in base_model.layers:
        if layer.name == "block5_conv1":
            trainable=True
        layer.trainable=trainable

    model=ks.models.Sequential()
    model.add(base_model)
    model.add(ks.layers.Flatten())
    model.add(ks.layers.Dropout(rate=0.25))
    model.add(ks.layers.Dense(1024,activation="relu"))
    model.add(ks.layers.Dense(512,activation="relu"))
    model.add(ks.layers.Dense(128,activation="relu"))
    model.add(ks.layers.Dense(64,activation="relu"))
    model.add(ks.layers.Dense(1,activation="sigmoid"))

    model.compile(optimizer="adam",loss=ks.losses.binary_crossentropy,metrics=["accuracy"])

    return model,base_model

In [12]:
model,base=create_model((224,224,3))

In [ ]:
ks.optimizers.adam()